In [1]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# ユーザが与えたトピックについて短いジョークをLLMに言わせるためのchain
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}") # プロンプトのテンプレート(invoke()を持っていて、このままchainに組み込めるところがふつうのstrと違う)
model = ChatOpenAI(model="gpt-3.5-turbo") # モデルにアクセスするためのAPI(モデルとの通信とかもこいつがやってそう)
output_parser = StrOutputParser() # モデルから帰ってきた出力をAIMessageからstrにしてくれるやつ、そしてchainに組み込めるやつ

chain = prompt | model | output_parser # これでchainを書けるの、イケてると思う

# chain.invoke({"topic": "ice cream"})

# 1. Prompt

プロンプトは`BasePromptTemplate`であり、テンプレート変数の辞書を受け取って`PromptValue`を返す。  
`PromptValue`は完成したプロンプトのラッパーで、`LLM`(strを受け取る)や`ChatModel`(メッセージのシーケンスを受け取る)に入力することができる。

プロンプトはstring出力と`BaseMessage`のシーケンスのどちらも出力するロジックを持っているので、両方のモデルと組み合わせることができる。

めっちゃ大事なこと書いてあった。

In [2]:
prompt_value = prompt.invoke({"topic": "ice cream"}) # `BasePromptTemplate`を`invoke()`すると`PromptValue`が返される
prompt_value # PromptValue形式になっていることがわかる

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

In [3]:
prompt_value.to_messages() # `PromptValue`から`BaseMessage`を生成するロジック

[HumanMessage(content='tell me a short joke about ice cream')]

In [4]:
prompt_value.to_string() # `PromptValue`からstrを生成するロジック

'Human: tell me a short joke about ice cream'

# 2. Model

`PromptValue`は`model`に渡される。このケースでは`ChatModel`であり、`BaseMessage`のシーケンスを受け取る。`ChatModel`の出力も`BaseMessage`になる。

In [5]:
# message = model.invoke(prompt_value) # PromptValueを受け取ってBaseMessageをかえす
# message

In [6]:
# modelが`LLM`なら出力はstr
from langchain_openai.llms import OpenAI

# llm = OpenAI(model="gpt-3.5-turbo-instruct") # `ChatModel`を呼び出すときは`ChatOpenAI()`, `LLM`を呼び出すときは`OpenAI()`
# llm.invoke(prompt_value) # チュートリアルと比べて、行頭のRobot:がないので、少し仕様が変わっていそう

# 3. Output parser

`model`の出力を`output_parser`に渡す。これは`BaseOutputParser`で、stringか`BaseMessage`を入力に受け取る。`StrOutputParser`は特別にどんな入力もstringに変える。

In [7]:
# output_parser.invoke(message) # 今回のは`StrOutputParser`なので、すべての入力はstrになる

# 4. パイプライン全体

1. ユーザが所望のトピックを`{"topic": "ice cream"}`として渡す。
1. `prompt`がユーザの入力を受け取り、`PromptValue`を出力する
1. `model`が`PromptValue`を受け取り、OpenAIのLLM modelに渡して出力を生成してもらう。モデルの出力は`ChatMessage`
1. `output_parser`が`ChatMessage`を受け取り、Pythonの文字列に変換する。

In [8]:
input = {"topic": "ice cream"}

prompt.invoke(input) # ChatPromptValueを返す
# > ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

In [9]:
# (prompt | model).invoke(input) # promptがChatPromptValueを返し、それがmodel(ChatModel, BaseMessageのシーケンスを受け取る)に入力され、AIMessageを返す
# > AIMessage(content="Why did the ice cream go to therapy?\nBecause it had too many toppings and couldn't cone-trol itself!")
